# Custom HOG for Ablation Studies


### Understanding Folders
###### train - this contains images the HOG features are trained on 
###### trainHOG - this contains the outputs after training the HOG features
###### test - this contains images that the model will be tested on
###### testHOG - this contains the images used to test the efficacy of our model


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import Custom_HOGFeatures
import csv
import numpy as np
import os

In [2]:
# Clear training
for entry in os.scandir("trainHOG"):
    if entry.name.endswith(".txt"):
        os.remove(entry.path)

In [3]:

# Clear testing
for entry in os.scandir("testHOG"):
    if entry.name.endswith(".txt"):
        os.remove(entry.path)

#### Generate Tester HOGS

# Ablation Studies

### Generate HOG list for training

In [39]:
# ① import the refactored module
import importlib, ablation_HOG as hogpipe
importlib.reload(hogpipe)                 # ② pull in any edits

# ③ build a config with your preferred overrides
cfg = (
    hogpipe.PipelineConfig(
        label_column="class",
        positive_labels={"person", "pedestrians"}
    )             
    .with_overrides(
        # ── HOG core ─────────────────────────────────────────────────────
        gradient="sobel",      # sobel | scharr | prewitt | roberts | dog
        unsigned=False,         # 0–180° bins (True) vs. 0–360° (False)
        bin_size=9,
        cell_size=8,
        block_norm="l2hys",    # l2 | l2hys | l1 | none
        phog_levels=0,         # 0 = off

        # ── pre-processing ──────────────────────────────────────────────
        color_space="gray",    # gray | rgb | lab | ycrcb
        gamma=1.0,
        gaussian_sigma=0.5,

        # ── dataset / I-O tweaks ────────────────────────────────────────
        max_images=5_000,
        save_descriptor=True,  # write *_fd.txt vectors
        annot_csv="PETA/3DPeS/archive/Label.txt",
        train_dir="train",
        out_dir="trainHOG",

        # TODO: THIS DOESN't WORJ, PETA IS TOTALLY DIFFERENT - NEED TO INTEGRATE IT
    )
)

# ④ run the extractor
hogpipe.run_pipeline(cfg)


Processed 500 images…
Processed 1000 images…
Processed 1500 images…
Processed 2000 images…
Processed 2500 images…
Processed 3000 images…
Processed 3500 images…
Processed 4000 images…
Processed 4500 images…
Processed 5000 images…
Finished extracting!


### Check if it all worked 


In [40]:

def count_files(directory):
    return sum(1 for item in os.listdir(directory) if os.path.isfile(os.path.join(directory, item)))

print(count_files("trainHOG"))

for entry in os.scandir("trainHOG"):
    if entry.name.endswith(".txt"):
        print(entry.path)
        print(np.loadtxt(entry.path, delimiter=','))

5000
trainHOG/1310_2008_006315_jpg.rf.a46c17189fa52e3ce0d882fc1d64d560.jpgF_fd.txt
[0.00309033 0.         0.00060578 ... 0.1020818  0.10786211 0.03279525]
trainHOG/519_000001_jpg.rf.d8208b654cf1d8cbff65a8e9fc0da035.jpgF_fd.txt
[0.32055518 0.32055518 0.15714291 ... 0.1410841  0.02834075 0.0167144 ]
trainHOG/3232_2007_001698_jpg.rf.7fae5ed610c57548d15bb4fb909911c1.jpgF_fd.txt
[0.01689641 0.0129961  0.0105647  ... 0.06475798 0.08681374 0.06851695]
trainHOG/301_2009_002612_jpg.rf.7f82b058b0893ca938fc39f7c8653159.jpgF_fd.txt
[0.08576496 0.01652066 0.08897709 ... 0.16462037 0.04475105 0.17073265]
trainHOG/3944_000190_jpg.rf.2c605a0760c1b1bcd4962dd4e7a9ab8d.jpgF_fd.txt
[0.16069126 0.32494059 0.15011077 ... 0.12629709 0.11308944 0.06830054]
trainHOG/99_2010_002400_jpg.rf.28673d065ba9aa369b3444dd0777dfbe.jpgF_fd.txt
[0.35350284 0.12732436 0.00882082 ... 0.03136571 0.03096339 0.04668066]
trainHOG/4149_000037_jpg.rf.662da4d12db51ef8a921fce15532f49c.jpgF_fd.txt
[1.66791156e-01 4.97292615e-02 9.538

#### Generate Tester HOGS

In [44]:
# -------------------------------------------------- ① reload module
import importlib, ablation_HOG as hogpipe
importlib.reload(hogpipe)           # pull in any edits

# -------------------------------------------------- ② build config (same values)
cfg = (
    hogpipe.PipelineConfig()        # start from defaults
    .with_overrides(
        # ── HOG core ─────────────────────────────────────────────────────
        gradient="roberts",
        unsigned=False,
        bin_size=9,
        cell_size=8,
        block_norm="l2hys",
        phog_levels=0,              # no PHOG

        # ── pre-processing ──────────────────────────────────────────────
        color_space="gray",
        gamma=1.0,
        gaussian_sigma=0.5,

        # ── dataset / I-O tweaks ────────────────────────────────────────
        annot_csv="test/_annotations.csv",
        train_dir="test",
        out_dir="testHOG",         # single output folder
        max_images=1_000,
        save_descriptor=True,       # write *_fd.txt
    )
)

# -------------------------------------------------- ③ run it
hogpipe.run_pipeline(cfg)


Processed 500 images…
Processed 1000 images…
Finished extracting!


In [45]:

def count_files(directory):
    return sum(1 for item in os.listdir(directory) if os.path.isfile(os.path.join(directory, item)))

print(count_files("testHOG"))

for entry in os.scandir("testHOG"):
    if entry.name.endswith(".txt"):
        print(entry.path)
        print(np.loadtxt(entry.path, delimiter=','))

1000
testHOG/511_2008_001811_jpg.rf.6a1e916abf01e742d0dbe74dba2359e3.jpgT_fd.txt
[0.         0.         0.         ... 0.23116146 0.34567001 0.12613857]
testHOG/639_2008_002716_jpg.rf.fe56eef743151962cac43018b5bfab9c.jpgT_fd.txt
[0.25317195 0.0031659  0.04090876 ... 0.19382608 0.24812102 0.21580796]
testHOG/904_2008_001039_jpg.rf.fa4c9fb4c1692efa99ce2a1a52a1d0db.jpgF_fd.txt
[0.0842886  0.05781227 0.18405269 ... 0.15823826 0.18435128 0.05829406]
testHOG/361_2007_003189_jpg.rf.cc869dd83dbdd3b7223c5dc4ab865471.jpgF_fd.txt
[0.01760723 0.01995092 0.01634472 ... 0.10353422 0.12272932 0.18139143]
testHOG/585_000070_jpg.rf.02d768e759ffe84c4c4213297517a935.jpgF_fd.txt
[0.10300865 0.05460313 0.041198   ... 0.05346965 0.01596469 0.02071072]
testHOG/749_2007_003118_jpg.rf.6b44d3afc7c9ca7d6b37cccf17e315d7.jpgT_fd.txt
[0.3287358  0.33384901 0.15167397 ... 0.24602795 0.25680026 0.1753903 ]
testHOG/382_2011_006843_jpg.rf.ec484544d5e90d950a986c43d8513fb4.jpgT_fd.txt
[0.24854973 0.27622128 0.27622128 ..

## Using Vanilla HOG() Function

### Generate Training HOGs

In [37]:
import os
import csv
import cv2
import numpy as np
from skimage.feature import hog
from pathlib import Path

numberoftrainingimages = 5000
output_dir = "trainHOG"
Path(output_dir).mkdir(parents=True, exist_ok=True)

num = 0
with open("train/_annotations.csv", mode="r", newline="") as file:
    csvFile = csv.DictReader(file)
    for lines in csvFile:
        num += 1
        if num > numberoftrainingimages:
            break

        img = cv2.imread(os.path.join("train", lines["filename"]))
        if img is None:
            continue

        crop_img = img[
            int(lines["ymin"]):int(lines["ymax"]),
            int(lines["xmin"]):int(lines["xmax"]),
        ]
        if crop_img.size == 0:
            continue

        img_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_gray, (64, 128), interpolation=cv2.INTER_LINEAR)

        fd, _ = hog(
            img_resized,
            orientations=9,
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2),
            visualize=True,
        )

        suffix = "T" if lines["class"].lower() in {"person", "pedestrians"} else "F"
        out_filename = f"{num}_{lines['filename']}{suffix}_fd.txt"
        out_path = Path(output_dir) / out_filename
        np.savetxt(out_path, fd, delimiter=",")


### Generate Tester HOGS

In [36]:
import os
import csv
import cv2
import numpy as np
from skimage.feature import hog
from pathlib import Path

numberoftestingimages = 3000
output_dir = "testHOG"
Path(output_dir).mkdir(parents=True, exist_ok=True)

num = 0
with open("test/_annotations.csv", mode="r", newline="") as file:
    csvFile = csv.DictReader(file)
    for lines in csvFile:
        num += 1
        if num > numberoftestingimages:
            break

        img = cv2.imread(os.path.join("test", lines["filename"]))
        if img is None:
            continue

        crop_img = img[
            int(lines["ymin"]):int(lines["ymax"]),
            int(lines["xmin"]):int(lines["xmax"]),
        ]
        if crop_img.size == 0:
            continue

        img_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        img_s = cv2.resize(img_gray, (64, 128), interpolation=cv2.INTER_LINEAR)

        fd, _ = hog(
            img_s,
            orientations=9,
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2),
            visualize=True,
        )

        suffix = "T" if lines["class"].lower() in {"person", "pedestrians"} else "F"
        filename = f"{num}_{lines['filename']}{suffix}_fd.txt"
        out_path = Path(output_dir) / filename
        np.savetxt(out_path, fd, delimiter=",")